In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import Model, layers
from keras.layers import Conv1D, MaxPooling1D, Conv1DTranspose, UpSampling1D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('data/comp-merged.csv')
df = df[['value_acc', 'value_met']]
scaler = StandardScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [ ]:
batch_size = 32
epochs = 100
window_size = 16

def windowed_dataset(data, window_size, batch_size):
    """
    Create a windowed dataset from the input data.

    Parameters:
    data (np.array): The input data array.
    window_size (int): The size of each window.
    batch_size (int): The size of each batch.

    Returns:
    tf.data.Dataset: The windowed dataset.
    """
    dataset = tf.data.Dataset.from_tensor_slices(data)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.map(lambda window: (window[:-1], window[1:]))
    dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

In [ ]:
# Split the dataframe into train and test sets
train, test = train_test_split(df, test_size=0.2, shuffle=False)

train_set = windowed_dataset(train, window_size, batch_size)
test_set = windowed_dataset(train, window_size, batch_size)

In [ ]:
def build_model():
    model = keras.Sequential([
        layers.Input(shape=(window_size, 2)),
        Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'),
        MaxPooling1D(pool_size=2),
        Conv1DTranspose(filters=128, kernel_size=3, padding='same', activation='relu'),
        UpSampling1D(size=2),
        Conv1DTranspose(filters=64, kernel_size=3, padding='same', activation='relu'),
        UpSampling1D(size=2),
        Conv1DTranspose(filters=32, kernel_size=3, padding='same', activation='relu'),
        UpSampling1D(size=2),
        Conv1DTranspose(filters=2, kernel_size=3, padding='same', activation='relu')
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
model.fit(train_set, epochs=epochs, validation_data=test_set)